# Calculations on DataFrames

files needed = none

Now that we understand DataFrames, let's look at basic calculations on DataFrames. We will cover more advanced calculation later. 

The big idea here is that **we never want to loop over the rows of a DataFrame to perform a calculation.**

Pandas gives us lots of ways to perform simple and complex computations in a DataFrame. When we use the pandas' tools, there is a lot of optimized code working in the background. When we loop over the DataFrame on our own, we lose all that fast code and our program grinds away slowly. 

In [11]:
import pandas as pd


A DataFrame naturally understands how to perform operations element-wise. For example, let's compute the share of consumption in GDP. We get started by loading up our little data dictionary and creating a DataFrame. 

In [12]:
data_dict = {'year': [1990, 1995, 2000, 2005 ], 'gdp':[5.9, 7.6, 10.2, 13.0], 'cons':[3.8, 4.9, 6.8, 8.7]}
data = pd.DataFrame(data_dict)   

In [13]:
data

,year,gdp,cons
0,1990,5.9,3.8
1,1995,7.6,4.9
2,2000,10.2,6.8
3,2005,13.0,8.7


The DataFrame currently has an uninformative "range" index. 

In [14]:
print(data.axes)

[RangeIndex(start=0, stop=4, step=1), Index(['year', 'gdp', 'cons'], dtype='object')]


We could do better if we wanted to improve readability and meaning.

In [15]:
data_wind = data.set_index('year')
data_wind

,gdp,cons
year,,
1990,5.9,3.8
1995,7.6,4.9
2000,10.2,6.8
2005,13.0,8.7


In [16]:
# This will divide cons by gdp: row by row.
data['cons_share'] = data['cons'] / data['gdp'] 

data

,year,gdp,cons,cons_share
0,1990,5.9,3.8,0.644068
1,1995,7.6,4.9,0.644737
2,2000,10.2,6.8,0.666667
3,2005,13.0,8.7,0.669231


Notice the left-hand side of the assignment. I am creating a new variable (column) in the DataFrame and assigning the consumption share to it.

In [17]:
# Oops, I wanted it in percentage
data['cons_share'] = data['cons_share']*100

data

,year,gdp,cons,cons_share
0,1990,5.9,3.8,64.406780
1,1995,7.6,4.9,64.473684
2,2000,10.2,6.8,66.666667
3,2005,13.0,8.7,66.923077


The ```+,-,/,*,**``` operators all work element wise. As we have seen, multiplying and dividing by scalar works fine, too.

In [18]:
temp = data['cons_share']**2

temp

0    4148.233266
1    4156.855956
2    4444.444444
3    4478.698225
Name: cons_share, dtype: float64

Recall that every column within a DataFrame is a series, and every series has an index.

In [19]:
gdp = data['gdp']
cons = data['cons']
cons_share = data['cons_share']

# Use the concat method to combine separate series into a DataFrame
data_v2 = pd.concat([gdp, cons, cons_share], axis=1)
data_v2

,gdp,cons,cons_share
0,5.9,3.8,64.406780
1,7.6,4.9,64.473684
2,10.2,6.8,66.666667
3,13.0,8.7,66.923077


Pandas calculations even work on series that are ordered differently, if they have the same index. Let's try sorting consumption in *descending* order.

In [20]:
cons = cons.sort_values(ascending=False)
cons

3    8.7
2    6.8
1    4.9
0    3.8
Name: cons, dtype: float64

Now, let's see what happens if we try to run a calculation on two series with different sorts.

In [21]:
print(cons_share, '\n\n')

cons_share_v2 = (cons / gdp)*100
print(cons_share_v2)

0    64.406780
1    64.473684
2    66.666667
3    66.923077
Name: cons_share, dtype: float64 


0    64.406780
1    64.473684
2    66.666667
3    66.923077
dtype: float64


Neat! This is **not** what would have happened had we tried to run a calculation on two spreadsheets with different sort orders.

### DataFrame methods for simple operations
DataFrame has many methods for computing various statistics on the data. Note that some of them take an axis argument: For example, you could compute `.sum()` across a row or a column. You have to tell pandas which one you want.

In [22]:
print(data)

# Sums 
print('Sum across columns')
print(data.sum(axis=1)) # summing across columns. Not terribly useful here.

print('\nSum across rows')
print(data.sum(axis=0)) # summing across rows. Cumulative GDP, consumption

print('\nSum up gdp')
print(data['gdp'].sum()) # Sum a single column. No axis necessary because this is a 1-D series.

# Means
print('\nMean of each column')
print(data.mean(axis=0)) 

print('\nMean gdp and cons')
print(data[['gdp', 'cons']].mean(axis=0)) # We could also omit the axis here as well


   year   gdp  cons  cons_share
0  1990   5.9   3.8   64.406780
1  1995   7.6   4.9   64.473684
2  2000  10.2   6.8   66.666667
3  2005  13.0   8.7   66.923077
Sum across columns
0    2064.106780
1    2071.973684
2    2083.666667
3    2093.623077
dtype: float64

Sum across rows
year          7990.000000
gdp             36.700000
cons            24.200000
cons_share     262.470207
dtype: float64

Sum up gdp
36.7

Mean of each column
year          1997.500000
gdp              9.175000
cons             6.050000
cons_share      65.617552
dtype: float64

Mean gdp and cons
gdp     9.175
cons    6.050
dtype: float64


What happens to a sum across columns if we set the index?

In [23]:
print('Sum across columns, range index')
print(data.sum(axis=1)) 

data_wind = data.set_index('year')
print('\n\nSum across columns, year index')
print(data_wind.sum(axis=1)) 

Sum across columns, range index
0    2064.106780
1    2071.973684
2    2083.666667
3    2093.623077
dtype: float64


Sum across columns, year index
year
1990    74.106780
1995    76.973684
2000    83.666667
2005    88.623077
dtype: float64


Try TAB completion to see the methods available or the [documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

Here are a few: `sum`, `mean`, `var`, `std`, `skew`, `rank`, `quantile`, `mode`, `min`, `max`, `kurtosis`, `cumsum`, `cumprod`...

These will be even more powerful once we learn how to group data within a DataFrame and compute statistics by group.

One very useful one...

In [24]:
# .describe() is a good place to start with a new data set.
print(data.describe())   

print('\n\n')  # Print a few blank lines.
print(data)


              year       gdp      cons  cons_share
count     4.000000   4.00000  4.000000    4.000000
mean   1997.500000   9.17500  6.050000   65.617552
std       6.454972   3.10309  2.157931    1.363750
min    1990.000000   5.90000  3.800000   64.406780
25%    1993.750000   7.17500  4.625000   64.456958
50%    1997.500000   8.90000  5.850000   65.570175
75%    2001.250000  10.90000  7.275000   66.730769
max    2005.000000  13.00000  8.700000   66.923077



   year   gdp  cons  cons_share
0  1990   5.9   3.8   64.406780
1  1995   7.6   4.9   64.473684
2  2000  10.2   6.8   66.666667
3  2005  13.0   8.7   66.923077


### Top Hat Practice Exercise: Calculations on DataFrames
Take a few minutes and try the following. Feel free to chat with those around you if you get stuck. I am here, too. 

1. Compute the mean of the consumption share for 1990 and 1995. You might try using `.loc[]` with two arguments `.loc[rows, cols]`

In [29]:
data
#data.set_index ('year', inplace = True)
#data

,gdp,cons,cons_share
year,,,
1990,5.9,3.8,64.406780
1995,7.6,4.9,64.473684
2000,10.2,6.8,66.666667
2005,13.0,8.7,66.923077


In [41]:
data.loc[(1990:1995, "cons_share").mean()]    #wrong format

SyntaxError: invalid syntax (4037310447.py, line 1)

In [42]:
data.loc['1990':'1995', 'cons_share'].mean()

64.44023193577164

In [37]:
print("the mean consumptiopn share for 1990 and 1995 is", format(data.loc[('1990':'1995', "cons_share"].mean()))

SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' (1498914372.py, line 1)

2. Try `desc = data.describe()` What is the return type?

In [38]:
desc = data.describe()
desc

,gdp,cons,cons_share
count,4.00000,4.000000,4.000000
mean,9.17500,6.050000,65.617552
std,3.10309,2.157931,1.363750
min,5.90000,3.800000,64.406780
25%,7.17500,4.625000,64.456958
50%,8.90000,5.850000,65.570175
75%,10.90000,7.275000,66.730769
max,13.00000,8.700000,66.923077
